In [1]:
file = '/labs/lab07data/DO_record_per_line.json'

In [2]:
idtexts = [[21545, u'en', u"The Divine Comedy: Dante's Journey to Freedom, Part 2 (Purgatorio)"], [7802, u'en', u'AE1110x: Introduction to Aeronautical Engineering'], [26228, u'es', u'Como ser Excelente Alumno - Udemy'], [196, u'es', u'Egiptolog\xeda (Egyptology)'], [12917, u'ru', u'\u041f\u0440\u0438\u043d\u0446\u0438\u043f\u044b \u044d\u0444\u0444\u0435\u043a\u0442\u0438\u0432\u043d\u043e\u0433\u043e \u0443\u043f\u0440\u0430\u0432\u043b\u0435\u043d\u0438\u044f \u0438\u043d\u0432\u0435\u0441\u0442\u0438\u0446\u0438\u043e\u043d\u043d\u044b\u043c\u0438 \u043f\u0440\u043e\u0435\u043a\u0442\u0430\u043c\u0438'], [21319, u'ru', u'\u041a\u043e\u043c\u043f\u0435\u0442\u0435\u043d\u0442\u043d\u043e\u0441\u0442\u043d\u043e-\u043e\u0440\u0438\u0435\u043d\u0442\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u0435 \u0432 \u0432\u044b\u0441\u0448\u0435\u043c \u043e\u0431\u0440\u0430\u0437\u043e\u0432\u0430\u043d\u0438\u0438']]

In [3]:
ids = [x[0] for x in idtexts]

In [4]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [5]:
import pandas as pd
import pickle
import json
import operator as op
import numpy as np
import re

In [6]:
df = spark.read.json(file).repartition(15).cache()

In [7]:
df.show(n=3)

In [8]:
re_words = re.compile(r'[\w\d]{2,}', re.U)
def split_lower(text):
    return re_words.findall(text.lower())

In [9]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
hashingTF = HashingTF(10000)

In [10]:
from pyspark.sql.types import Row

In [11]:
def row2prop(prop):
    return lambda row: row[prop]

In [12]:
rdd_words = df.rdd.map(lambda row: Row(id=row.id, lang=row.lang, words=re_words.findall(row.desc.lower())))

In [13]:
rdd_words.map(row2prop('words')).take(1)

In [14]:
tf = hashingTF.transform(rdd_words.map(row2prop('words')))

In [15]:
idf = IDF(minDocFreq=2).fit(tf)

In [16]:
tfidf = idf.transform(tf)

In [17]:
tfidf.cache()

In [18]:
rdd_with_tfidf = rdd_words.zip(tfidf).map(lambda t: Row(tfidf=t[1], **t[0].asDict())).cache()

In [19]:
rdd_with_tfidf.toDF().show()

In [20]:
targets = rdd_with_tfidf.filter(lambda row: row.id in ids).collect()

In [21]:
len(targets)

In [22]:
def cossim(vec1, vec2):
    return vec1.dot(vec2)/(vec1.norm(2) * vec2.norm(2))

In [23]:
results =[]
for target in targets:
    vec = target.tfidf
    lang = target.lang
    id = target.id
    res = rdd_with_tfidf\
        .filter(lambda row: row.lang == lang and row.id != id)\
        .map(lambda row: (cossim(vec, row.tfidf), row.id))\
        .collect()
    res = sorted(res, reverse=True)
    results.append(res)

In [24]:
fin = dict([(targets[i].id, [v for k,v in results[i][:10]]) for i in range(len(targets))])

In [25]:
with open('lab07.json', 'w') as fw:
    fw.write(json.dumps(fin))

In [26]:
with open('lab07s.json', 'w') as fw:
    fw.write(json.dumps(fin))

In [27]:
!cat lab07.json

In [28]:
sc.stop()